In [77]:
import networkx as nx
import matplotlib.pyplot as plt
import dzcnapy_plotlib as dzcnapy
import csv

In [78]:
with open("email-Eu-core.csv") as infile:
    csv_reader = csv.reader(infile)
    DG = nx.DiGraph(csv_reader)
#print(DG.nodes(data = True))

In [79]:
#print(DG.edges(data = True))

In [80]:
with open("email-Eu-core-department-labels.csv") as infile:
    csv_reader = csv.reader(infile)
    mydict = {rows[0]: {'department': int(rows[1])} for rows in csv_reader}

In [81]:
#print(mydict)

In [82]:
nx.set_node_attributes(DG, mydict)
#print(DG.nodes(data = True))

In [83]:
nx.write_graphml(DG, "emails_dep.graphml")

In [84]:
DG_only_dep = nx.DiGraph()
for node in DG.nodes(data = True):
    label, attr_dict = node
    DG_only_dep.add_node(attr_dict["department"])
    
print(nx.info(DG_only_dep))

Name: 
Type: DiGraph
Number of nodes: 42
Number of edges: 0
Average in degree:   0.0000
Average out degree:   0.0000


In [85]:
for edge in DG.edges(data=True):
    #print(edge)
    source, target, edg_attr = edge
    source_dep = mydict[source]['department']
    targ_dep = mydict[target]['department']
    if DG_only_dep.has_edge(source_dep, targ_dep) == False:
        DG_only_dep.add_edge(source_dep, targ_dep, weight = 1)
    else:
        edges = DG_only_dep.edges
        w = edges[source_dep, targ_dep]['weight']
        DG_only_dep.add_edge(source_dep, targ_dep, weight = w+1)
print(nx.info(DG_only_dep))

Name: 
Type: DiGraph
Number of nodes: 42
Number of edges: 1243
Average in degree:  29.5952
Average out degree:  29.5952


In [86]:
print(DG_only_dep.edges(data=True))

[(1, 1, {'weight': 539}), (1, 15, {'weight': 44}), (1, 39, {'weight': 24}), (1, 3, {'weight': 8}), (1, 0, {'weight': 19}), (1, 7, {'weight': 41}), (1, 14, {'weight': 30}), (1, 16, {'weight': 4}), (1, 31, {'weight': 14}), (1, 20, {'weight': 2}), (1, 25, {'weight': 18}), (1, 19, {'weight': 3}), (1, 36, {'weight': 109}), (1, 34, {'weight': 37}), (1, 37, {'weight': 12}), (1, 35, {'weight': 23}), (1, 6, {'weight': 11}), (1, 38, {'weight': 16}), (1, 23, {'weight': 7}), (1, 2, {'weight': 9}), (1, 29, {'weight': 2}), (1, 9, {'weight': 21}), (1, 24, {'weight': 3}), (1, 17, {'weight': 3}), (1, 11, {'weight': 5}), (1, 4, {'weight': 52}), (1, 21, {'weight': 11}), (1, 12, {'weight': 1}), (1, 5, {'weight': 11}), (1, 27, {'weight': 1}), (1, 13, {'weight': 17}), (1, 26, {'weight': 17}), (1, 10, {'weight': 18}), (1, 40, {'weight': 2}), (1, 41, {'weight': 4}), (1, 28, {'weight': 4}), (1, 8, {'weight': 2}), (1, 32, {'weight': 1}), (1, 22, {'weight': 1}), (1, 30, {'weight': 1}), (21, 21, {'weight': 640}),

In [87]:
nx.write_graphml(DG_only_dep, "dep_only.graphml")
# TODO: remove self loops and add their weight as attribute of nodes

In [90]:
print(DG['0'])

{'1': {}, '316': {}, '146': {}, '268': {}, '581': {}, '221': {}, '18': {}, '734': {}, '178': {}, '380': {}, '0': {}, '459': {}, '215': {}, '250': {}, '148': {}, '73': {}, '498': {}, '226': {}, '101': {}, '218': {}, '377': {}, '17': {}, '74': {}, '177': {}, '248': {}, '103': {}, '560': {}, '309': {}, '88': {}, '5': {}, '297': {}, '313': {}, '223': {}, '238': {}, '368': {}, '266': {}, '222': {}, '283': {}, '6': {}, '64': {}, '166': {}}


In [94]:
print("density of emails:", str(nx.density(DG)))
print("density of deps:", str(nx.density(DG_only_dep)))

density of emails: 0.025342411448732432
density of deps: 0.721835075493612


In [96]:
nx.average_shortest_path_length(DG)

NetworkXError: Graph is not weakly connected.

In [98]:
nx.diameter(DG)

NetworkXError: Found infinite path length because the digraph is not strongly connected

In [100]:
nx.radius(DG)

NetworkXError: Found infinite path length because the digraph is not strongly connected

In [101]:
nx.eccentricity(DG)

NetworkXError: Found infinite path length because the digraph is not strongly connected